In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from datetime import datetime
import numpy as np
import math

client_id = "6538920d435d4008b7a494fe07783e46"
client_secret = "7d8bf7518fd64e528602d51e0838d579"


In [19]:
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)

In [20]:
sp_query = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [21]:
# Set up query with credentials
sp = spotipy.Spotify(
            client_credentials_manager=SpotifyClientCredentials(
            client_id=client_id,
            client_secret=client_secret))


In [22]:
url_jazz = 'spotify:playlist:37i9dQZF1DX5S7hzwhDSyV'
url_hiphop = 'spotify:playlist:37i9dQZF1DX1lHW2vbQwNN'

In [23]:
def analyse_playlist(url):
    """Retrieves all songs from an identified playlist, and takes the ID, song, album, artist 
    and puts this all into a DataFrame"""

   # SONG NAMES

    offset=0
    name = []

    while True:
        response = sp.playlist_tracks(url,
                                      offset=offset,
                                      fields=['items.track.name,total'])

        name.append(response["items"])
        offset = offset + len(response['items'])
        
        if len(response['items']) == 0:
            break

    name_list = [b["track"]["name"] for a in name for b in a]
    len(name_list)
    
    

    # ALBUM

    offset=0
    album = []

    while True:
        response = sp.playlist_tracks(url,
                                      offset=offset,
                                      fields=['items.track.album.name,total'])

        album.append(response["items"])
        offset = offset + len(response['items'])
        
        if len(response['items']) == 0:
            break

    album_list = [b["track"]["album"]["name"] for a in album for b in a]

    
    
   # ARTIST

    offset=0
    artist = []

    while True:
        response = sp.playlist_tracks(url,
                                      offset=offset,
                                      fields=['items.track.album.artists.name,total'])

        artist.append(response["items"])
        offset = offset + len(response['items'])
        
        if len(response['items']) == 0:
            break

    artist_list = [b["track"]["album"]["artists"][0]["name"] for a in artist for b in a]

    
    # ID
    
    offset = 0
    identifier = []

    while True:
        response = sp.playlist_tracks(url,
                                      offset=offset,
                                      fields=['items.track.id,total'])

        identifier.append(response["items"])
        offset = offset + len(response['items'])
        
        if len(response['items']) == 0:
            break

    identifier_list= [b["track"]["id"] for a in identifier for b in a]
    len(identifier_list)

    #Get audio features
    features = [sp.audio_features(identifier) for identifier in identifier_list]
    
    # Get each invidividual feature
    danceability = [(b["danceability"]) for a in features for b in a]    
    mode = [(b["mode"]) for a in features for b in a]
    energy = [(b["energy"]) for a in features for b in a]
    key = [(b["key"]) for a in features for b in a]        
    loudness = [(b["loudness"]) for a in features for b in a]       
    speechiness = [(b["speechiness"]) for a in features for b in a]
    acousticness = [(b["acousticness"]) for a in features for b in a]        
    instrumentalness = [(b["instrumentalness"]) for a in features for b in a] 
    liveness = [(b["liveness"]) for a in features for b in a]
    valence = [(b["valence"]) for a in features for b in a]        
    tempo = [(b["tempo"]) for a in features for b in a] 
    duration_ms = [(b["duration_ms"]) for a in features for b in a] 
    identifier_ = [(b["id"]) for a in features for b in a] 
    
    ## DataFrame (saved with current time)

    df = pd.DataFrame({"Song name": name_list, "Artist": artist_list, "Album": album_list, "ID": identifier_list})
    df_2 = pd.DataFrame({"Danceability":danceability,
                         "Mode":mode,
                         "Energy":energy,
                         "Key":key,
                         "Loudness":loudness,
                         "Speechiness":speechiness,
                         "Acousticness":acousticness,
                         "Instrumentalness":instrumentalness,
                         "Liveness":liveness,
                         "Valence":valence,
                         "Tempo":tempo,
                         "Duration (ms)": duration_ms,
                         "ID_CHECK":identifier_
                               })

    df_combined = df_2.join(df)
    
    df_combined.to_excel("df" + datetime.now().strftime(format="%f") + ".xlsx")

    return df_combined.tail()

In [24]:
analyse_playlist(url_hiphop)
analyse_playlist(url_jazz)

,Danceability,Mode,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration (ms),ID_CHECK,Song name,Artist,Album,ID
95,0.482,1,0.0961,3,-14.024,0.0360,0.930,0.014,0.0491,0.319,61.297,322960,57Qv0Ls4xEB3UblVSeCbUJ,I Guess I'll Hang My Tears Out To Dry,Dexter Gordon,Ballads,57Qv0Ls4xEB3UblVSeCbUJ
96,0.362,1,0.1960,5,-24.658,0.0450,0.895,0.876,0.6650,0.133,118.535,496893,47ncHNZLs7ZHvDkCi9tvun,Blame It On My Youth,Keith Jarrett Trio,The Cure,47ncHNZLs7ZHvDkCi9tvun
97,0.508,1,0.4820,2,-13.728,0.0805,0.509,0.040,0.1470,0.649,144.252,553067,3xEAZQNPiwucPdGagdoD5j,I'll Remember April,Clifford Brown,Clifford Brown And Max Roach At Basin Street (...,3xEAZQNPiwucPdGagdoD5j
98,0.514,0,0.4090,5,-10.341,0.0627,0.810,0.770,0.2340,0.439,139.331,193160,2F0cAwp8zmQ8McgBzjFWO0,Stars Fell On Alabama,Toots Thielemans,The Real... Toots Thielemans,2F0cAwp8zmQ8McgBzjFWO0
99,0.510,1,0.3230,0,-9.856,0.0459,0.835,0.000,0.1680,0.673,174.841,237333,1QpoiqlsaMLOfxfcnjIpuy,You Took Advantage Of Me,Benny Carter,Cosmopolite: The Oscar Peterson Verve Sessions,1QpoiqlsaMLOfxfcnjIpuy


In [25]:
# Label the class of each genre, jazz = 0, hiphop = 1
labelled_jazz = pd.read_excel("JAZZ_.xlsx", index_col=0, usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
labelled_hiphop = pd.read_excel("HIPHOP.xlsx", index_col=0, usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
labelled_jazz["Class"] = 0
labelled_hiphop["Class"] = 1

# Combine to create a full dataframe
full_data = labelled_jazz.append(labelled_hiphop[:1861], ignore_index=True)

# Scale between 0 and 1
full_data["Key"] = (full_data["Key"] / full_data["Key"].max())
full_data["Tempo"] = (full_data["Tempo"] / full_data["Tempo"].max())
full_data["Loudness"] = (full_data["Loudness"] / full_data["Loudness"].min())

full_data_random = full_data.sample(frac=1)
full_data_random

FileNotFoundError: [Errno 2] No such file or directory: 'JAZZ_.xlsx'

In [26]:
# Get x training data
x_train = full_data_random[0:3000]

# Target values
y_train = x_train["Class"].values

# Remove target values from x_data
x_train = x_train.drop("Class", axis=1)
x_train

NameError: name 'full_data_random' is not defined

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from IPython.core.display import Image, display

knn = KNeighborsClassifier(n_neighbors=61)

#cross_validate
cv_scores = cross_val_score(knn, x_train, y_train, cv=10)
knn.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [ ]:
x_test = full_data_random[3000:]

# Randomise the data
x_test = x_test.sample(frac=1)

y_test = x_test["Class"].values

x_test = x_test.drop("Class", axis=1)
x_test

In [ ]:
predictions = knn.predict(x_test)
predictions

In [ ]:
accuracy_score = knn.score(x_test, y_test)
print("Accuracy score = ""{:.2f}".format(accuracy_score))

In [ ]:
url_test = 'spotify:playlist:1mjYVGCwhXZVxoNWBBLlL0'
analyse_playlist(url_test)

In [8]:
labelled_test = pd.read_excel("TEST.xlsx", index_col=0, usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
labelled_test["Key"] = (labelled_test["Key"] / labelled_test["Key"].max())
labelled_test["Tempo"] = (labelled_test["Tempo"] / labelled_test["Tempo"].max())
labelled_test["Loudness"] = (labelled_test["Loudness"] / labelled_test["Loudness"].min())

FileNotFoundError: [Errno 2] No such file or directory: 'TEST.xlsx'

In [9]:
predictionsTest = knn.predict(labelled_test)
predictionsTest

NameError: name 'knn' is not defined

In [10]:
targets = [1,0,1,0,1,1,1,1]
results = pd.DataFrame(targets,predictionsTest,columns=["Predictions"])
results.index.rename('Targets', inplace=True)
results

NameError: name 'predictionsTest' is not defined